<a href="https://colab.research.google.com/github/jdcanado/teste/blob/master/join_cep_setor_censitario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pycep_correios




In [20]:
!pip install geopandas

In [21]:
!pip install Nominatim


In [2]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from pycep_correios import get_address_from_cep, WebService, exceptions
from geopy.geocoders import Nominatim

uri = "https://raw.githubusercontent.com/jdcanado/teste/master/CEP_todas_bases.csv"
 
dados = pd.read_csv(uri, delimiter=",")
df = dados.head(50)

for row in df.index:
   cep = '' + df['CEP'][row]
   
   while True:
      try:
          endereco = get_address_from_cep(cep, webservice=WebService.VIACEP)    
          break
      except exceptions.InvalidCEP as eic:
          print(eic)
          
      except exceptions.CEPNotFound as ecnf:
          print(ecnf)
          
      except exceptions.ConnectionError as errc:
          print(errc)
          
      except exceptions.Timeout as errt:
          print(errt)
          
      except exceptions.HTTPError as errh:
          print(errh)
          
      except exceptions.BaseException as e:
          print(e)          
          
      except:
          print('Erro ao tentar localizar o cep ' + cep)

   while True:
      try:             
          geolocator = Nominatim(user_agent="test_app")
          location = geolocator.geocode(endereco['logradouro'] + ", " + endereco['cidade'] + " - " + endereco['bairro'])
          geolocation_info_df = pd.DataFrame({'cep': [cep], 'Logradouro': [endereco['logradouro']], 'bairro': endereco['bairro'], 'cidade': endereco['cidade'], 'uf': endereco['uf']})
          geolocation_info_df['coordinate_point'] = Point(location.longitude, location.latitude)
          census_sector_gpd = gpd.read_file('/content/drive/My Drive/join_cep_setor_censitario/dataset/35SEE250GC_SIR.shp')
          geolocation_info_df['census_code'] = geolocation_info_df['coordinate_point'].map(lambda x: census_sector_gpd.loc[census_sector_gpd.contains(x), 'CD_GEOCODI'].values).str[0].astype('int64')
          print('CEP: ' + str(geolocation_info_df['cep'].values[0]) + ' ---> Código censo: ' + str(geolocation_info_df['census_code'].values[0]) + ' ---> Longitude: ' + str(location.longitude) + ' ---> Latitude: ' + str(location.latitude))      
          break
      except:
          print('Erro ao tentar obter a localização para o cep ' + cep)
          break


CEP: 01001000 ---> Código censo: 355030878000004 ---> Longitude: -46.633080956332904 ---> Latitude: -23.550389799999998
CEP: 01001001 ---> Código censo: 355030878000004 ---> Longitude: -46.633080956332904 ---> Latitude: -23.550389799999998
CEP: 01001010 ---> Código censo: 355030878000004 ---> Longitude: -46.6338028 ---> Latitude: -23.5512564
CEP: 01001901 ---> Código censo: 355030878000004 ---> Longitude: -46.6340301 ---> Latitude: -23.5513772
CEP: 01002000 ---> Código censo: 355030878000005 ---> Longitude: -46.6361174 ---> Latitude: -23.5479884
CEP: 01002001 ---> Código censo: 355030878000005 ---> Longitude: -46.6361174 ---> Latitude: -23.5479884
CEP: 01002010 ---> Código censo: 355030878000006 ---> Longitude: -46.63642875 ---> Latitude: -23.54761425
CEP: 01002020 ---> Código censo: 355030866000003 ---> Longitude: -46.6385296 ---> Latitude: -23.5459758
CEP: 01003000 ---> Código censo: 355030878000005 ---> Longitude: -46.6384205 ---> Latitude: -23.5486085
CEP: 01003001 ---> Código cens

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
